# Network Recovery Using Netplan (Ubuntu Server)

## Context

During the initial setup of a headless Ubuntu Server, the system booted correctly but **had no network connectivity**.  
This prevented package installation, remote access (SSH), and normal server operation.

The issue occurred after:
- Installing Ubuntu Server without active internet
- Connecting the server through a managed switch
- No DHCP lease being assigned automatically

---

## Symptoms

- `apt update` failed with: 

Temporary failure resolving 'archive.ubuntu.com'

- `dhclient` was not available
- `nmcli` was not installed
- Network interface appeared as `UP` but without IPv4 address
- `/etc/netplan/` directory existed but was **empty**

---

## Diagnosis

1. Verified physical connectivity:
 ```bash
 ip a
```
 Interface enp5s0 was detected but had no IPv4 address.

2. Confirmed link state:
 ```bash
sudo ip link set enp5s0 up
```

3. Verified Layer 2 / Layer 3 behavior:

+ No DHCP lease

+ No default gateway

+ DNS resolution failing

4. Confirmed that NetworkManager was not installed

+ System was using Netplan + systemd-networkd

## Solution

1. Create Netplan configuration manually

Since /etc/netplan/ was empty, a new configuration file was created:
```bash
sudo nano /etc/netplan/01-netcfg.yaml
```

In [ ]:
network:
  version: 2
  renderer: networkd
  ethernets:
    enp5s0:
      dhcp4: true

2. Fix permissions (security requirement)

Netplan requires restrictive permissions:
```bash
sudo chmod 600 /etc/netplan/01-netcfg.yaml
```

3. Apply configuration
```bash
sudo netplan apply
```

Warnings about permissions were resolved after fixing file mode.

### Validation (Layered Verification)


Layer 3 – IP Assignment
```bash
ip a
```

Result:

+ IPv4 address successfully assigned via DHCP

Default Gateway

```bash
ip route
```
Local Network Connectivity
```bash
ping -c 3 xxx.xxx.xxx.1
```

External Connectivity
```bash
ping -c 3 8.8.8.8
```
DNS Resolution
```bash
ping -c 3 google.com
```

All tests passed successfully.

## Result

+ Network connectivity fully restored

+ Server obtained dynamic IP via DHCP

+ DNS resolution working

+ Internet access confirmed

+ System ready for package installation and remote administration


## Key Takeaways

+ Netplan may ship without a default configuration

+ A server can be fully operational at L2 but unusable at L3/L7

+ Always validate networking by layers

+ Understanding Netplan + systemd-networkd is critical for headless servers

+ This recovery process is a core sysadmin skill, not a beginner task


